In [1]:
# import torch

# batch_size=1
# seq_len=3
# input_size=4
# hidden_size=2

# cell=torch.nn.RNNCell(input_size=input_size, hidden_size=hidden_size)

# dataset=torch.randn(seq_len,batch_size,input_size)
# hidden=torch.zeros(batch_size,hidden_size)

# for idx,x_input in enumerate(dataset):
#     print("="*20,idx,"="*20)
#     print("input size:",x_input.shape)
    
#     hidden=cell(x_input,hidden)
    
#     print("output size:",hidden.shape)
#     print(hidden)
    

==================== 0 ====================
input size: torch.Size([1, 4])
output size: torch.Size([1, 2])
tensor([[-0.8730,  0.2915]], grad_fn=<TanhBackward0>)
==================== 1 ====================
input size: torch.Size([1, 4])
output size: torch.Size([1, 2])
tensor([[-0.9684,  0.7902]], grad_fn=<TanhBackward0>)
==================== 2 ====================
input size: torch.Size([1, 4])
output size: torch.Size([1, 2])
tensor([[-0.9559,  0.4415]], grad_fn=<TanhBackward0>)


In [2]:
# import torch

# batch_size=1
# seq_len=3
# input_size=4
# hidden_size=2
# num_laryers=1

# cell=torch.nn.RNN(input_size=input_size,hidden_size=hidden_size,num_layers=num_laryers)

# inputs=torch.randn(seq_len,batch_size,input_size)
# hidden=torch.zeros(num_laryers,batch_size,hidden_size)

# out,hidden=cell(inputs,hidden)

# print("output size:",out.shape)
# print("output",out)
# print("hidden size",hidden.shape)
# print("hidden",hidden)


output size: torch.Size([3, 1, 2])
output tensor([[[ 0.6526,  0.7850]],

        [[-0.9475, -0.3880]],

        [[-0.3591, -0.2576]]], grad_fn=<StackBackward0>)
hidden size torch.Size([1, 1, 2])
hidden tensor([[[-0.3591, -0.2576]]], grad_fn=<StackBackward0>)


In [1]:
import torch
import torch.nn as nn

input_size = 4
hidden_size = 4
batch_size = 1

idx2char = ['e', 'h', 'l', 'o']
x_data = [1, 0, 2, 2, 3]
y_data = [3, 1, 2, 3, 2]
one_hot_lookup = [[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]]
x_one_hot = [one_hot_lookup[x] for x in x_data]

inputs = torch.Tensor(x_one_hot).view(-1, batch_size, input_size)
labels = torch.LongTensor(y_data).view(-1,1)


In [2]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, batch_size):
        super(RNNModel, self).__init__()
        
        self.batch_size = batch_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.rnncell = nn.RNNCell(input_size = self.input_size, 
                                  hidden_size = self.hidden_size)
        
    def forward(self, x, hidden):
        hidden = self.rnncell(x, hidden)
        return hidden
    
    def init_hidden(self):
        return torch.zeros(self.batch_size, self.hidden_size)

                
rnnNet = RNNModel(input_size, hidden_size, batch_size)        

In [3]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnnNet.parameters(),lr=0.1)

In [4]:
for epoch in range(30):
    loss = 0
    optimizer.zero_grad()
    hidden = rnnNet.init_hidden()
    print('Predicted string: ', end='')
    for input, label in zip(inputs, labels):
        hidden = rnnNet(input, hidden)
        loss += criterion(hidden, label)
        _, idx = hidden.max(dim=1)
        print(idx2char[idx.item()], end='')
    loss.backward()
    optimizer.step()
    print(', Epoch [%d/15] loss=%.4f' % (epoch+1, loss.item()))

Predicted string: oooeo, Epoch [1/15] loss=6.7143
Predicted string: ooooo, Epoch [2/15] loss=5.3658
Predicted string: ooloo, Epoch [3/15] loss=4.5355
Predicted string: ohloo, Epoch [4/15] loss=3.9396
Predicted string: ohlol, Epoch [5/15] loss=3.5215
Predicted string: ohlol, Epoch [6/15] loss=3.1565
Predicted string: ohlol, Epoch [7/15] loss=2.8225
Predicted string: ohlol, Epoch [8/15] loss=2.5455
Predicted string: ohlol, Epoch [9/15] loss=2.3116
Predicted string: ohlol, Epoch [10/15] loss=2.1624
Predicted string: ohlol, Epoch [11/15] loss=2.0806
Predicted string: ohlol, Epoch [12/15] loss=2.0274
Predicted string: ohlol, Epoch [13/15] loss=1.9830
Predicted string: ohlol, Epoch [14/15] loss=1.9437
Predicted string: ohlol, Epoch [15/15] loss=1.9108
Predicted string: ohlol, Epoch [16/15] loss=1.8848
Predicted string: ohlol, Epoch [17/15] loss=1.8645
Predicted string: ohlol, Epoch [18/15] loss=1.8480
Predicted string: ohlol, Epoch [19/15] loss=1.8336
Predicted string: ohlol, Epoch [20/15] l

In [1]:
import torch

# parameters
num_class = 4
input_size = 4
hidden_size = 8
embedding_size = 10
num_layers = 2
batch_size = 1
seq_len = 5

# 准备数据集
idx2char = ['e', 'h', 'l', 'o']
x_data = [[1, 0, 2, 2, 3]]  # (batch, seq_len)
y_data = [3, 1, 2, 3, 2]    # (batch * seq_len)

inputs = torch.LongTensor(x_data)   # Input should be LongTensor: (batchSize, seqLen)
labels = torch.LongTensor(y_data)   # Target should be LongTensor: (batchSize * seqLen)

In [5]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.emb = torch.nn.Embedding(input_size, embedding_size)
        self.rnn = torch.nn.RNN(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, num_class)
        
    def forward(self, x):
        hidden = torch.zeros(num_layers, x.size(0), hidden_size)
        x = self.emb(x)
        x, _ = self.rnn(x, hidden)
        x = self.fc(x)
        return x.view(-1, num_class)
    

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.emb = torch.nn.Embedding(input_size, embedding_size)
        self.rnn = torch.nn.RNN(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, num_class)

    def forward(self, x):
        hidden = torch.zeros(num_layers, x.size(0), hidden_size)
        x = self.emb(x)  # (batch, seqLen, embeddingSize)
        x, _ = self.rnn(x, hidden)  # 输出(𝒃𝒂𝒕𝒄𝒉𝑺𝒊𝒛𝒆, 𝒔𝒆𝒒𝑳𝒆𝒏, hidden_size)
        x = self.fc(x)  # 输出(𝒃𝒂𝒕𝒄𝒉𝑺𝒊𝒛𝒆, 𝒔𝒆𝒒𝑳𝒆𝒏, 𝒏𝒖𝒎𝑪𝒍𝒂𝒔𝒔)
        return x.view(-1, num_class)  # reshape to use Cross Entropy: (𝒃𝒂𝒕𝒄𝒉𝑺𝒊𝒛𝒆×𝒔𝒆𝒒𝑳𝒆𝒏, 𝒏𝒖𝒎𝑪𝒍𝒂𝒔𝒔)


In [6]:
net = Model()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)


In [7]:
for epoch in range(15):
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    print(outputs.shape)
    _, idx = outputs.max(dim=1)
    idx = idx.data.numpy()
    print('Predicted: ', ''.join([idx2char[x] for x in idx]), end='')
    print(', Epoch [%d/15] loss = %.3f' % (epoch + 1, loss.item()))


torch.Size([5, 4])
Predicted:  lelee, Epoch [1/15] loss = 1.471
torch.Size([5, 4])
Predicted:  lolol, Epoch [2/15] loss = 1.083
torch.Size([5, 4])
Predicted:  lolol, Epoch [3/15] loss = 0.826
torch.Size([5, 4])
Predicted:  oolol, Epoch [4/15] loss = 0.629
torch.Size([5, 4])
Predicted:  ohlol, Epoch [5/15] loss = 0.471
torch.Size([5, 4])
Predicted:  ohlol, Epoch [6/15] loss = 0.342
torch.Size([5, 4])
Predicted:  ohlol, Epoch [7/15] loss = 0.241
torch.Size([5, 4])
Predicted:  ohlol, Epoch [8/15] loss = 0.169
torch.Size([5, 4])
Predicted:  ohlol, Epoch [9/15] loss = 0.119
torch.Size([5, 4])
Predicted:  ohlol, Epoch [10/15] loss = 0.084
torch.Size([5, 4])
Predicted:  ohlol, Epoch [11/15] loss = 0.060
torch.Size([5, 4])
Predicted:  ohlol, Epoch [12/15] loss = 0.043
torch.Size([5, 4])
Predicted:  ohlol, Epoch [13/15] loss = 0.032
torch.Size([5, 4])
Predicted:  ohlol, Epoch [14/15] loss = 0.024
torch.Size([5, 4])
Predicted:  ohlol, Epoch [15/15] loss = 0.018
